In [ ]:
# Basic python import
import os
import sys
import yaml
import matplotlib.pyplot as plt
import pandas as pd

# Pytorch
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# Our custom module
sys.path.append('../')
import data_processing
import models
from evaluation import *
import submission

# Config

Ma proposition : 
Avoir un Notebook template associé à des paramètres par défaut. 
Ensuite pour chaque expérience on le duplique, et on change les valeurs des paramètres que l'on souhaite.

In [ ]:
with open('../configs/default_params.yaml') as default_params_file:
      default_params = yaml.safe_load(default_params_file)
params = default_params
params

data_dir = os.path.join('..','data')

DEBUG = False

torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

device

In [ ]:
# Changer de paramètres
params["train"]["batch_size"] = 1
params["train"]

# If necessary, pre-process raw data (to do once on each machine)

In [ ]:
from data_processing.preproc import create_masks_as_tiff, preprocess_images_and_masks

# create_masks_as_tiff(data_dir)


# Create Dataset and Dataloader

In [ ]:
from data_processing.dataset import CustomDataset, DebugCustomDataset
import torchvision.transforms as transforms

transform = transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((512,512))
])

if DEBUG :
    train_dataset = DebugCustomDataset(root_dir = os.path.join('..','data'), transform=transform)
else : 
    train_dataset = CustomDataset(root_dir = os.path.join('..','data'), transform=transform)

train_dl = DataLoader(train_dataset, batch_size=params["train"]["batch_size"], shuffle=True)

# Train

In [ ]:
from models.unet import UNet

MODEL = UNet(num_classes=1)
lr = 1e-3
optimizer = optim.Adam(MODEL.parameters(), lr=lr)
loss = nn.MSELoss()
optimizer

In [ ]:
from train.train import main_train, main_train_batch1

loss_list = main_train_batch1(model=MODEL.to(device), loss_fn=loss, optimizer=optimizer, n_epochs=100, dataset=train_dataset, device=device)

In [ ]:
import numpy as np

def show_image_and_mask(image,mask,cmaps):
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 32))
    hybr = image[0, :, :]/2 + mask[0, :, :]

    ax[0].imshow(image.T)
    ax[0].axis('off')
    ax[0].set_title('IMAGE')
    ax[1].imshow(hybr.T,cmap=cmaps)
    ax[1].axis('off')
    ax[1].set_title('MASK ON IMAGE')
    plt.show()

image = train_dataset[0][0]
mask = train_dataset[0][2]

print(image.shape)
print(mask.shape)

#show_image_and_mask(image,mask,"gray")
#show_image_and_mask(image,MODEL(torch.unsqueeze(image, dim=0).to(device)).cpu().detach(),"gray")

### Save Model

In [ ]:
import time
torch.save(MODEL, f"../model_save/save_{time.gmtime(time.time())}")

# Evaluation

Running model on validation dataset

In [ ]:
import gc
gc.collect()

In [ ]:
# val_pred = {"val_images":[], "val_masks":[], "val_organs":[], "val_y_preds":[]}
# for k, (val_image, val_organ, val_mask) in enumerate(train_dl):
#     print(k)
#     val_pred['val_images'].append(val_image)
#     val_pred['val_masks'].append(val_mask)
#     val_pred['val_organs'].append(val_organ)
    
#     gc.collect()
#     print(torch.cuda.memory_allocated(0),
#         torch.cuda.memory_reserved(0),
#         torch.cuda.max_memory_reserved(0),)
#     pred = MODEL(val_image.to(device))
#     val_pred['val_y_preds'].append(pred.cpu)

In [ ]:
#Validation images, masks and organs
val_preds = {0:{"val_images":[], "val_masks":[], "val_organs":[], "val_y_preds":[]}} #Only 1 fold

for l, batch in enumerate(train_dl):
    (val_images, val_organs, val_masks) = next(iter(train_dl))
    val_mask_preds = MODEL(val_images.to(device)).cpu().detach()
    print(f'VAL_Y_PREDS shape: {val_mask_preds.shape}, VAL_Y_PREDS dtype: {val_mask_preds.dtype}')
    print(f'val_images shape: {val_images.shape}, val_masks shape: {val_masks.shape}, val_organs shape: {len(val_organs)}')
    for k in range(len(val_images)):
        # Cast from Tensorflow to Numpy
        val_preds[0]['val_images'].append(val_images[k].numpy())
        val_preds[0]['val_masks'].append(val_masks[k].numpy().astype(np.uint8))
        val_preds[0]['val_organs'].append(val_organs[k])
        val_preds[0]['val_y_preds'].append(val_mask_preds[k].numpy())

In [ ]:
gc.collect()
print(torch.cuda.memory_allocated(0),
      torch.cuda.memory_reserved(0),
    torch.cuda.max_memory_reserved(0),)

In [ ]:
from evaluation.evaluation import get_y_true_y_pred

IoU_Folds = dict()
for fold, v in val_preds.items():
    IoUs, IoUsOrgans = get_y_true_y_pred(v)
    IoU_Folds[fold] = {
        'IoUs': IoUs,
        'IoUsOrgans': IoUsOrgans,
    }

In [ ]:
from evaluation.evaluation import plot_iou_by_threshold

# Global Mean Intersection over Union at Threshold
for fold, v in IoU_Folds.items():
    print('=' * 80)
    print(f'FOLD {fold}')
    print('=' * 80)
    v['threshold_best'] = plot_iou_by_threshold(v['IoUs'], f'all_{fold}')

In [ ]:
for fold, v in IoU_Folds.items():
    print('=' * 80)
    print(f'FOLD {fold}')
    print('=' * 80)
    
    percentiles = [0.01, 0.05, 0.10, 0.25, 0.40, 0.50, 0.60, 0.75, 0.90, 0.95, 0.99]
    s = v['IoUs'][v['threshold_best']]
    
    display(pd.Series(s).describe(percentiles=percentiles).apply(lambda v: f'{v:.2f}').to_frame(name='Value').T)

In [ ]:
for fold, v in IoU_Folds.items():
    print('=' * 80)
    print(f'FOLD {fold}')
    print('=' * 80)
    plt.figure(figsize=(12,8))
    pd.Series(v['IoUs'][v['threshold_best']]).plot(kind='hist')
    plt.title('IoU Distribution at Best Threshold', size=24)
    plt.grid()
    plt.xlabel('Threshold', size=16)
    plt.ylabel('Count', size=16)
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.xlim(0,1)
    plt.show()

We can also look at some of the results

In [ ]:
from evaluation.evaluation import plot_validation_predictions

for fold, v in IoU_Folds.items():
    print('=' * 80)
    print(f'FOLD {fold}')
    print('=' * 80)
    threshold_best = IoU_Folds[fold]['threshold_best']
    plot_validation_predictions(val_preds[fold], threshold_best, 2, IoUs)

# Submission

In [ ]:
from submission import submission

test_dataset = None # TO BUILD
submission.make_submission(MODEL, test_dataset, best_threshold)